Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [1]:
subscription_id = 'a89228ac-8bf4-4646-9d2c-442b1cb5d622'
resource_group = 'lauri-ml'
aml_workspace = 'lauri-ml'
cluster_name = 'bert-gpu'
# azure_dataset_name = 'Azure Services Dataset'
# azure_dataset_path = 'azure-service-classifier/data'
# azure_dataset_descr = 'Dataset containing azure related posts on Stackoverflow'
experiment_name = 'bert-classifier-test'

In [2]:
from azureml.core import Workspace #, Dataset, Experiment
# from azureml.core.runconfig import RunConfiguration
# from azureml.widgets import RunDetails
# from azureml.train.dnn import TensorFlow
from model import TFBertForMultiClassification
from transformers import BertTokenizer
import tensorflow as tf

### Check Azure Machine Learning Python SDK version

This tutorial requires version 1.0.69 or higher. Let's check the version of the SDK:

In [3]:
import azureml.core

print("Azure Machine Learning Python SDK version:", azureml.core.VERSION)

Azure Machine Learning Python SDK version: 1.0.85


## Connect To Workspace

In [ ]:
# aml_workspace = Workspace(
#     subscription_id=subscription_id, resource_group=resource_group, workspace_name=aml_workspace
# )
# aml_workspace.write_config()

In [4]:
# from azureml.core.authentication import InteractiveLoginAuthentication
# interactive_auth = InteractiveLoginAuthentication()
# workspace = Workspace.from_config(auth=interactive_auth)

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: lauri-ml
Azure region: westeurope
Subscription id: a89228ac-8bf4-4646-9d2c-442b1cb5d622
Resource group: lauri-ml


## Choose Compute Target

#### If the compute target has already been created, then you (and other users in your workspace) can directly run this cell.

In [12]:
# compute_target = workspace.compute_targets[cluster_name]

#### You can also use a local compute target:

In [14]:
local_runcfg = RunConfiguration()
local_runcfg.environment.python.user_managed_dependencies = True
compute_target = local_runcfg.target

In [7]:
# datastore_name = 'tfworld'

In [7]:
# from azureml.core import Datastore, Dataset

#### If the datastore has already been registered, then you (and other users in your workspace) can directly run this cell.

In [8]:
# datastore = workspace.datastores[datastore_name]

#### If the dataset has already been registered, then you (and other users in your workspace) can directly run this cell.

In [16]:
# azure_dataset = workspace.datasets[azure_dataset_name]

## Perform Experiment

Now that we have our compute target, dataset, and training script working locally, it is time to scale up so that the script can run faster. We will start by creating an [experiment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py). An experiment is a grouping of many runs from a specified script. All runs in this tutorial will be performed under the same experiment. 

In [21]:
experiment = Experiment(workspace, name=experiment_name)

In [27]:
# run.download_files(prefix='../outputs/model')

# If you haven't finished training the model then just download pre-made model from datastore
# datastore.download('./',prefix="azure-service-classifier/model")

#### Instantiate the model

Next step is to import our model class and instantiate fine-tuned model from the model file.

In [29]:
def encode_example(text, max_seq_length):
    # Encode inputs using tokenizer
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_seq_length
    )
    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
    attention_mask = [1] * len(input_ids)
    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    input_ids = input_ids + ([0] * padding_length)
    attention_mask = attention_mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
    
    return input_ids, attention_mask, token_type_ids

In [30]:
labels = ['azure-web-app-service', 'azure-storage', 'azure-devops', 'azure-virtual-machine', 'azure-functions']
# Load model and tokenizer
# loaded_model = TFBertForMultiClassification.from_pretrained('azure-service-classifier/model', num_labels=len(labels))
loaded_model = TFBertForMultiClassification.from_pretrained('../outputs/model', num_labels=len(labels))
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
print("Model loaded from disk.")

Model loaded from disk.


#### Define prediction function

Using the model object we can interpret new questions and predict what Azure service they talk about. To do that conveniently we'll define **predict** function.

In [31]:
# Prediction function
def predict(question):
    input_ids, attention_mask, token_type_ids = encode_example(question, 128)
    predictions = loaded_model.predict({
        'input_ids': tf.convert_to_tensor([input_ids], dtype=tf.int32),
        'attention_mask': tf.convert_to_tensor([attention_mask], dtype=tf.int32),
        'token_type_ids': tf.convert_to_tensor([token_type_ids], dtype=tf.int32)
    })
    prediction = labels[predictions[0].argmax().item()]
    probability = predictions[0].max()
    result = {
        'prediction': str(labels[predictions[0].argmax().item()]),
        'probability': str(predictions[0].max())
    }
    print('Prediction: {}'.format(prediction))
    print('Probability: {}'.format(probability))

#### Experiement with our new model

Now we can easily test responses of the model to new inputs. 
*  **ACTION**: Invent yout own input for one of the 5 services our model understands: 'azure-web-app-service', 'azure-storage', 'azure-devops', 'azure-virtual-machine', 'azure-functions'.

In [32]:
# Route question
predict("How can I specify Service Principal in devops pipeline when deploying virtual machine")

Prediction: azure-devops
Probability: 0.2559393048286438


In [33]:
# Now more tricky cae - the opposite
predict("How can virtual machine trigger devops pipeline")

Prediction: azure-devops
Probability: 0.2656690180301666
